In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv("Train_B_Tree.csv")

In [5]:
class Node:
    def __repr__(self):
        return "<Node Object with attribute "+str(self.attr)+" and bound "+str(self.bound)+" and decision "+str(self.decision)+">"
    def __init__(self, *args, **kwargs):
        try:
            self.parent = kwargs['parent']
        except:
            self.parent = None
        try:
            self.left = kwargs['left']
        except:
            self.left = None
        try:
            self.right = kwargs['right']
        except:
            self.right = None
        try:
            self.decision = kwargs['decision']
        except:
            self.decision = None
        try:
            self.attr = kwargs['attr']
        except:
            self.attr = None
        try:
            self.bound = kwargs['bound']
        except:
            self.bound = None

In [6]:
def regressiontree(df):
    train=df.sample(frac=0.7) #random state is a seed value
    test=df.drop(train.index)
    list_attr = train.columns.to_numpy()[:-1].tolist()
    NodeList=[]
    root = Node()
    split(train, root, 'left', list_attr, NodeList)
    sqerr = 0
    for i in train.index:
        start = root.left
        #print(type(start))
        try:
            while(not start.decision):
                if(train[start.attr][i]<=start.bound):
                    start = start.left
                if(train[start.attr][i]>start.bound):
                    start = start.right
        except:
            #print(start)
            pass
        e_val = start.decision - train['csMPa'][i]
        #print(e_val)
        sqerr = sqerr + e_val**2
    print(sqerr)
    return


In [7]:
def LS_ERROR(train, list_attr):
    input_train = train[list_attr]
    if(len(input_train.drop_duplicates())<=1):
        return 0
    else:
        #t = type(train['csMPa'][0])
        m = train['csMPa'].mean()
        ERROR = 0
        #print(len(train))
        for i in train.index:
            t=type(i)
            ERROR = ERROR + (train['csMPa'][i] - m)**2
        return ERROR

In [8]:
def WEIGHTED_LS(left, right, list_attr):
    lls = LS_ERROR(left, list_attr)
    rls = LS_ERROR(right, list_attr)
    return (lls*len(left) + rls*len(right))

In [9]:
def split(root_train, parent, side, list_attr, NodeList):
    root_train = root_train.iloc[:, np.random.permutation(len(root_train.columns))]
    a = Node(parent=parent)
    if(side == 'left'):
        parent.left = a
    else:
        parent.right = a
    NodeList.append(a)
    input_train = root_train[list_attr]
    if(LS_ERROR(root_train, list_attr) == 0):
        a.decision = root_train['csMPa'].mean()
        return
    MIN_LS_ERROR = np.inf
    for attribute in list_attr:
        valid_set = root_train[attribute].unique().tolist()
        valid_set.sort()
        if(len(valid_set)>=2):
            for i in valid_set[:-1]:
                left_temp = root_train[root_train[attribute]<=i]
                right_temp = root_train[root_train[attribute]>i]
                LS = WEIGHTED_LS(left_temp, right_temp, list_attr)
                if(LS<MIN_LS_ERROR):
                    #print(LS[0])
                    left = left_temp
                    right = right_temp
                    attr = attribute
                    bound = i
                    MIN_LS_ERROR = LS
    #print(MIN_LS_ERROR, attr, bound)
    a.attr = attr
    a.bound = bound
    split(left, a, 'left', list_attr, NodeList)
    split(right, a, 'right', list_attr, NodeList)
    return

In [10]:
#print(df[df['age']<=3])
if __name__ == "__main__":
    for i in range(10):
        regressiontree(df)
        
    

14484.935749999993


KeyboardInterrupt: 